In [1]:
import os
import requests
import json
from dotenv import load_dotenv
import pandas as pd
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
from pymongo import MongoClient
from pymongo import GEOSPHERE
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
from folium import CircleMarker

In [81]:
# for this scenario we are infering that its a Berlin based company
#checked the web and Berlin has a good enviroment and i like the city:
#"https://startupgenome.com/article/rankings-2021-top-30-plus-runners-up"
#"https://blog.honeypot.io/assets/images/startup2.png"
# "https://github.com/Chelechepe/geopython/blob/master/data/berlin-neighbourhoods.geojson"
# "https://upload.wikimedia.org/wikipedia/commons/6/68/Berlin%2C_administrative_divisions_%28%2Bdistricts_%2Bboroughs_-pop%29_-_de_-_colored.svg"

In [3]:
# Berlin, we get an initial location to set the map and locate where the
# main startups are located, we will be doing a heat map to set out desired
# for the city of Berlin we select the the coordinate location just to set the map starting position. 
inicial_lat = 52.5377398
inicial_long = 13.3546392

In [4]:
berlin_map = folium.Map(location = [inicial_lat, inicial_long], zoom_start=10)
berlin_map

In [5]:
# ok now lets get all the companies that have Berling offices

In [6]:
client = MongoClient("localhost:27017")

In [7]:
client.list_database_names()

['Ironhack', 'admin', 'config', 'local']

In [8]:
# we have conected to the Ironhack server and now we want to locate 
# the "companies" collection to display in our map.

In [9]:
db = client["Ironhack"]

In [10]:
db.list_collection_names()

['Test', 'myneigh', 'restaurants', 'companies', 'myrest']

In [11]:
companies = db.get_collection("companies")

In [12]:
companies.find_one()

{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
 'name': 'Wetpaint',
 'permalink': 'abc2',
 'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
 'homepage_url': 'http://wetpaint-inc.com',
 'blog_url': 'http://digitalquarters.net/',
 'blog_feed_url': 'http://digitalquarters.net/feed/',
 'twitter_username': 'BachelrWetpaint',
 'category_code': 'web',
 'number_of_employees': 47,
 'founded_year': 2005,
 'founded_month': 10,
 'founded_day': 17,
 'deadpooled_year': 1,
 'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
 'alias_list': '',
 'email_address': 'info@wetpaint.com',
 'phone_number': '206.859.6300',
 'description': 'Technology Platform Company',
 'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
 'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
 'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and monetize audiences for di

In [13]:
# im going to display the keys so that its easier to locate
# the location of the information that i need.

In [14]:
companies.find_one().keys()

dict_keys(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url', 'blog_url', 'blog_feed_url', 'twitter_username', 'category_code', 'number_of_employees', 'founded_year', 'founded_month', 'founded_day', 'deadpooled_year', 'tag_list', 'alias_list', 'email_address', 'phone_number', 'description', 'created_at', 'updated_at', 'overview', 'image', 'products', 'relationships', 'competitions', 'providerships', 'total_money_raised', 'funding_rounds', 'investments', 'acquisition', 'acquisitions', 'offices', 'milestones', 'video_embeds', 'screenshots', 'external_links', 'partners'])

In [15]:
companies.find_one()["offices"][0].keys()

dict_keys(['description', 'address1', 'address2', 'zip_code', 'city', 'state_code', 'country_code', 'latitude', 'longitude'])

In [16]:
# now we do a query with a list so that we can get the filtered 
# information that we need

In [17]:
query = {"offices.city": "Berlin"}
projection = {"_id":0,"name":1,"offices.city":1, "country_code":1,
              "offices.latitude":1, "offices.longitude":1,"office.city":1}

list(companies.find(query, projection))[:5]

[{'name': 'headr',
  'offices': [{'city': 'Berlin',
    'latitude': 52.5013449,
    'longitude': 13.4109071},
   {'city': 'Hannover', 'latitude': None, 'longitude': None}]},
 {'name': 'Plazes',
  'offices': [{'city': 'Berlin',
    'latitude': 52.523402,
    'longitude': 13.398694}]},
 {'name': 'GameDuell',
  'offices': [{'city': 'San Francisco', 'latitude': None, 'longitude': None},
   {'city': 'Berlin', 'latitude': 52.5131826, 'longitude': 13.3941489}]},
 {'name': 'Qiro',
  'offices': [{'city': 'Berlin',
    'latitude': 52.502633,
    'longitude': 13.441005}]},
 {'name': '99designs',
  'offices': [{'city': 'San Francisco',
    'latitude': 37.7955307,
    'longitude': -122.4005983},
   {'city': 'Collingwood', 'latitude': -37.8026587, 'longitude': 144.9868546},
   {'city': 'Berlin', 'latitude': 52.4986199, 'longitude': 13.4469031}]}]

In [18]:
# we get 94 companies that have offices in berling but 
# our locations have lat and lon for all their other offices and we 
# would need to get rid of this information. 

In [19]:
len(list(companies.find(query, projection)))

94

In [20]:
# first lets save it into a variable so that we can proceed to filter
# unwnated information.

In [21]:
berlin_company_dirty = list(companies.find(query, projection))

In [22]:
berlin_company_dirty

[{'name': 'headr',
  'offices': [{'city': 'Berlin',
    'latitude': 52.5013449,
    'longitude': 13.4109071},
   {'city': 'Hannover', 'latitude': None, 'longitude': None}]},
 {'name': 'Plazes',
  'offices': [{'city': 'Berlin',
    'latitude': 52.523402,
    'longitude': 13.398694}]},
 {'name': 'GameDuell',
  'offices': [{'city': 'San Francisco', 'latitude': None, 'longitude': None},
   {'city': 'Berlin', 'latitude': 52.5131826, 'longitude': 13.3941489}]},
 {'name': 'Qiro',
  'offices': [{'city': 'Berlin',
    'latitude': 52.502633,
    'longitude': 13.441005}]},
 {'name': '99designs',
  'offices': [{'city': 'San Francisco',
    'latitude': 37.7955307,
    'longitude': -122.4005983},
   {'city': 'Collingwood', 'latitude': -37.8026587, 'longitude': 144.9868546},
   {'city': 'Berlin', 'latitude': 52.4986199, 'longitude': 13.4469031}]},
 {'name': 'VZnet Netzwerke',
  'offices': [{'city': 'Berlin',
    'latitude': 52.5290751,
    'longitude': 13.4161514}]},
 {'name': 'Buongiorno',
  'office

In [23]:
berlin_company_dirty[0]["offices"]

[{'city': 'Berlin', 'latitude': 52.5013449, 'longitude': 13.4109071},
 {'city': 'Hannover', 'latitude': None, 'longitude': None}]

In [24]:
berlin_company_dirty[0]["name"]

'headr'

In [25]:
berlin_clean = []
for i in berlin_company_dirty:
    for j in i['offices']:
        if j['city'] == 'Berlin':
            city = j['city']
            lat = j['latitude']
            lon = j['longitude']
            name = i['name']
            type_ = ({"typepoint": 
                      {"type": "Point", 
                         "coordinates": [j['latitude'], j['longitude']]}})
            
            berlin_clean.append({"name":name, "lat":lat, "long":lon, "type":type_})
berlin_clean

[{'name': 'headr',
  'lat': 52.5013449,
  'long': 13.4109071,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [52.5013449, 13.4109071]}}},
 {'name': 'Plazes',
  'lat': 52.523402,
  'long': 13.398694,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [52.523402, 13.398694]}}},
 {'name': 'GameDuell',
  'lat': 52.5131826,
  'long': 13.3941489,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [52.5131826, 13.3941489]}}},
 {'name': 'Qiro',
  'lat': 52.502633,
  'long': 13.441005,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [52.502633, 13.441005]}}},
 {'name': '99designs',
  'lat': 52.4986199,
  'long': 13.4469031,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [52.4986199, 13.4469031]}}},
 {'name': 'VZnet Netzwerke',
  'lat': 52.5290751,
  'long': 13.4161514,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [52.5290751, 13.4161514]}}},
 {'name': 'Buongiorno',
  'lat': 52.488331,
  'long': 13.393718,
  'type': {'t

In [26]:
# now we have a list of the office locations for 
# the companies that are in Berlin, now we can make a dataframe
# of this companies

In [27]:
len(berlin_clean) # can see that we have kept all the the companies we filtered

94

In [28]:
df_berlin = pd.DataFrame(berlin_clean)
df_berlin

,name,lat,long,type
0,headr,52.501345,13.410907,"{'typepoint': {'type': 'Point', 'coordinates':..."
1,Plazes,52.523402,13.398694,"{'typepoint': {'type': 'Point', 'coordinates':..."
2,GameDuell,52.513183,13.394149,"{'typepoint': {'type': 'Point', 'coordinates':..."
3,Qiro,52.502633,13.441005,"{'typepoint': {'type': 'Point', 'coordinates':..."
4,99designs,52.498620,13.446903,"{'typepoint': {'type': 'Point', 'coordinates':..."
...,...,...,...,...
89,Springer,52.478717,13.311778,"{'typepoint': {'type': 'Point', 'coordinates':..."
90,Idealo,52.532547,13.408999,"{'typepoint': {'type': 'Point', 'coordinates':..."
91,Webtrekk,52.527926,13.381354,"{'typepoint': {'type': 'Point', 'coordinates':..."
92,Bigpoint,NaN,NaN,"{'typepoint': {'type': 'Point', 'coordinates':..."


In [29]:
# out our final filtered companies we have 17 out 94 that have no geo location
# so we will exclude them form the search.

In [30]:
df_berlin.isna().sum()

name     0
lat     17
long    17
type     0
dtype: int64

In [31]:
# we will use a df copy to make sure that we wont run into an issue

In [32]:
df_berlincopy = df_berlin.copy()

In [33]:
df_berlincopy.dropna(subset=['lat', 'long'],inplace=True)

In [34]:
df_berlincopy.isna().sum()

name    0
lat     0
long    0
type    0
dtype: int64

In [35]:
len(df_berlincopy)

77

In [36]:
# now we can make this dataframe into a heat map to define 
# our location of where we should have the companie venue.  

In [37]:
# this creates a feature group, so the layer will have this name:
berlin_group = folium.FeatureGroup(name="Berlin")

In [38]:
# this will creat a heat map that will display the areas where there is more congregation of companies.
HeatMap(data=df_berlincopy[["lat", "long"]], radius=15,
       gradient = {0.4: 'yellow', 0.65: 'orange', 1: 'red'}).add_to(berlin_group)

In [39]:
berlin_group.add_to(berlin_map)

In [40]:
# now that we have the map we can see that the district of "Mitte" 
# is the perfect location start searching for the companies venue

In [41]:
berlin_map

In [42]:
# the geolocation for the distric of Mitte is the following

In [43]:
mitte_lat = 52.53211713308012 
mitte_long = 13.370612229900534

In [44]:
mitte_map = folium.Map(location = [mitte_lat, mitte_long], zoom_start=10)
folium.Marker([mitte_lat, mitte_long]).add_to(mitte_map)
folium.Circle([mitte_lat, mitte_long], radius=500).add_to(mitte_map)
folium.Circle([mitte_lat, mitte_long], radius=1000).add_to(mitte_map)
folium.Circle([mitte_lat, mitte_long], radius=3000).add_to(mitte_map)

In [45]:
# created a map with marker of mittens with radius of 1km,5km and 10km,
# this will allow for us to see all what is within a range of 

In [46]:
mitte_map

In [47]:
# now we need to get all the markers of the different businesses in the area:

In [48]:
load_dotenv()

True

In [49]:
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
client_key = os.getenv("API_KEY")

In [50]:
# the next function is to facilitate the extraction of an API query:

In [51]:
def query4sqr (query, location, radius):

    if " " in query.strip():
           query_ok = query.strip().replace(" ","%20")
    else:
           query_ok = query
    url = f"https://api.foursquare.com/v3/places/search?query={query_ok}&ll={str(location[0]).strip()}%2C{str(location[1]).strip()}&radius={radius}&limit=50"
    headers =  {
    "Accept": "application/json",
    "Authorization": f"{client_key}" # tengo que validar si poner la llave en la funcion es factible.
    }
    response = requests.get(url, headers=headers)
    
    if response:
        response=response.json()['results']
        new_list = [ {"name" : i["name"],
                      "lat"  : i["geocodes"]["main"]["latitude"],
                      "long" : i["geocodes"]["main"]["longitude"],
                      "type" : {"typepoint":
                                  {"type": "Point",
                                   "coordinates": [lat, lon]}}} for i in response]
    return new_list

In [52]:
# we query on foursquare for venues that we want to see if they are around the area that we are 
# to buy our companies location. 

In [53]:
bar = query4sqr("bar",[52.53211713308012, 13.370612229900534],3000)

In [54]:
starbucks = query4sqr("starbucks",[52.53211713308012, 13.370612229900534],3000)

In [55]:
school = query4sqr("school",[52.53211713308012, 13.370612229900534],3000)

In [56]:
pet_grooming = query4sqr(" Pet Grooming Service",[52.53211713308012, 13.370612229900534],3000)

In [57]:
basketball_court = query4sqr("basketball_court",[52.53211713308012, 13.370612229900534],3000)

In [58]:
vegan = query4sqr("vegan",[52.53211713308012, 13.370612229900534],3000)

In [59]:
design_studio = query4sqr("Design Studio",[52.53211713308012, 13.370612229900534],3000)

In [60]:
# now  we create a dataframe for each list of venues so that we can display them on the map. 

In [61]:
df_bar = pd.DataFrame(data=bar)

In [62]:
df_starbucks = pd.DataFrame(data=starbucks)

In [63]:
df_school = pd.DataFrame(data=school)

In [64]:
df_grooming = pd.DataFrame(data=pet_grooming)

In [65]:
df_basket = pd.DataFrame(data=basketball_court)

In [66]:
df_vegan = pd.DataFrame(data=vegan)

In [67]:
df_design = pd.DataFrame(data=design_studio)

In [68]:
# with the dataframes ready we can now create all the markers and add them to the mitte_map

In [69]:
from folium.plugins import MarkerCluster

In [70]:
bar_group = folium.FeatureGroup(name="Bar")

for index, row in df_bar.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="red",
            prefix="fa",
            icon="beer",
            icon_color = "white")
    
    bar_group.add_child(Marker(**venue, icon=icon))
    mitte_map.add_child(bar_group)

In [71]:
starbucks_group = folium.FeatureGroup(name="Starbucks")

for index, row in df_starbucks.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="green",
            prefix="fa",
            icon="coffee",
            icon_color = "white")
    
    starbucks_group.add_child(Marker(**venue, icon=icon))
    mitte_map.add_child(starbucks_group)

In [72]:
school_group = folium.FeatureGroup(name="Schools")

for index, row in df_school.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="blue",
            prefix="fa",
            icon="graduation-cap",
            icon_color = "white")
    
    school_group.add_child(Marker(**venue, icon=icon))
    mitte_map.add_child(school_group)

In [73]:
grooming_group = folium.FeatureGroup(name="Pet_Grooming")

for index, row in df_grooming.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="orange",
            prefix="fa",
            icon="paw",
            icon_color = "white")
    
    grooming_group.add_child(Marker(**venue, icon=icon))
    mitte_map.add_child(grooming_group)

In [74]:
basket_group = folium.FeatureGroup(name="Basket")

for index, row in df_basket.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="orange",
            prefix="fa",
            icon="fa-dribbble",
            icon_color = "white")
    
    basket_group.add_child(Marker(**venue, icon=icon))
    mitte_map.add_child(basket_group)

In [75]:
vegan_group = folium.FeatureGroup(name="Vegan_Restaurant")

for index, row in df_vegan.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="purple",
            prefix="fa",
            icon="leaf",
            icon_color = "white")
    
    vegan_group.add_child(Marker(**venue, icon=icon))
    mitte_map.add_child(vegan_group)

In [76]:
design_group = folium.FeatureGroup(name="Design_Studio")

for index, row in df_design.iterrows():
    
    venue = {"location": [row["lat"], row["long"]], "tooltip": row["name"]}
    
    icon = Icon (
            color="darkgreen",
            prefix="fa",
            icon="desktop",
            icon_color = "white")
    
    design_group.add_child(Marker(**venue, icon=icon))
    mitte_map.add_child(design_group)

In [77]:
# once the markers where created we can create a layer control to be able to select the

In [78]:
folium.LayerControl(collapsed=False).add_to(mitte_map)

In [79]:
mitte_map

In [80]:
# we couldnt comply with all the requirements that some employees have
# if we where to have proximity to the airport the number of other venues 
# would decrease proportionaly to the inverse of the distance
# the closest stadium is the "Mercedez Benz Arena" which is 20km away 
# if we where closer to the Stadium we would have to be 10 km away from startup locations